<a href="https://colab.research.google.com/github/iwatake2222/colaboratory_study/blob/master/DL_tutorial/DL_tutorial_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learningアプリケーション開発 (7) TensorFlow Lite with Python on Raspberry Pi and Edge TPU
Post-training quantizationしたTensorFlow LiteモデルをEdge TPU上で動かしてみる

https://qiita.com/iwatake2222/items/6aeab468c326ecc21563


# 環境設定
It's better to use tensorflow 1.15 because it supports full integer post-training quantization.

You can still use tensorflow 2.x although it doesn't support full integer post-training quantization (fp32 to int8 conversion ops will be added to input/output), and the converted model may not work on Edge TPU though.
In case you want to use Tensorflow 2.x, you need to use tf-nightly(Tensorflow2.2). Otherwise the following error happend when converting to tflite

```
ValueError: Failed to parse the model: /tensorflow-2.1.0/python3.6/tensorflow_core/lite/python/optimize/_tensorflow_lite_wrap_calibration_wrapper.so: undefined symbol: _ZTIN10tensorflow6DeviceE.
```

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls

# Install EdgeTPU
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt -y update
!sudo apt -y install edgetpu

# Use Tensorflow 2.2.0, instead of tensorflow-2.1.0rc1
# %tensorflow_version 2.x
# !pip3 uninstall -y tensorflow
# !pip3 install tf-nightly

# モデル変換 (Keras(H5) -> Quantized Tensorflow Lite Model(tflite))

In [0]:
%tensorflow_version 1.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np

print(tf.__version__)

# Download Keraas model
# https://drive.google.com/open?id=1H7zKhgxdS6eiaMzxsyON9Wtne-pVnjXL
!wget -O conv_mnist.h5 "https://drive.google.com/uc?export=download&id=1OLR1n5Pq0CGPz7Bw5pad-fvYsgCnKvHh"

## Prepara dataset generator for calibration
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train = x_train / 255.0
x_train = x_train.astype(np.float32)
num_calibration_images = 100
calibration_indexes   = np.random.choice(x_train.shape[0], num_calibration_images, replace=False)
def representative_dataset_gen():
  for i in range(num_calibration_images):
    yield [x_train[calibration_indexes[i: i + 1]]]


# Convert
# loaded_model = tf.keras.models.load_model("conv_mnist.h5")
# converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter = tf.lite.TFLiteConverter.from_keras_model_file("conv_mnist.h5")

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.experimental_new_converter = True   # will be no need in the future

tflite_model = converter.convert()
open("conv_mnist_quant.tflite", "wb").write(tflite_model)

!edgetpu_compiler conv_mnist_quant.tflite

# モデル変換 (Tensorflow Saved Model(pb) -> Quantized Tensorflow Lite Model(tflite))

In [0]:
%tensorflow_version 1.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np

print(tf.__version__)

# Download Saved model
!wget -O conv_mnist_saved_model.tar.gz "https://drive.google.com/uc?export=download&id=1T0_2UYERZkTQnBZ4ocfnT7j1u0XJ2xKm"
!tar zxvf conv_mnist_saved_model.tar.gz
saved_model_dir = "./conv_mnist_saved_model"

## Prepara dataset generator for calibration
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train = x_train / 255.0
x_train = x_train.astype(np.float32)
num_calibration_images = 100
calibration_indexes   = np.random.choice(x_train.shape[0], num_calibration_images, replace=False)
def representative_dataset_gen():
  for i in range(num_calibration_images):
    yield [x_train[calibration_indexes[i: i + 1]]]


# Convert
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.experimental_new_converter = True   # will be no need in the future

tflite_model = converter.convert()
open("conv_mnist_quant.tflite", "wb").write(tflite_model)

!edgetpu_compiler conv_mnist_quant.tflite

# TFLiteモデルによる推論テスト
may not work on Tensorflow 1.15

In [0]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np


# Read input image
!wget -O 4.jpg  "https://drive.google.com/uc?export=download&id=1-3yb3qCrN8M6Bdj7ZZ9UMjONh34R2W_A" 
img = cv2.imread("4.jpg")
cv2_imshow(img)

# Pre process
## グレースケール化、リサイズ、白黒反転、価範囲を0～255 -> 0.0～1.0
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
img = 255 - img
# img = img / 255.
# img = img.astype(np.float32)
img = img.astype(np.uint8)
input_tensor = img.reshape(1, img.shape[0], img.shape[1], 1)
input_tensor = tf.convert_to_tensor(input_tensor)


# Load model
interpreter = tf.lite.Interpreter(model_path="conv_mnist_quant.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# set input tensor
interpreter.set_tensor(input_details[0]['index'], input_tensor)

# Inference
interpreter.invoke()

scores = interpreter.get_tensor(output_details[0]['index'])
result = np.argmax(scores[0])
print("predicted number is {} [{:.2f}]".format(result, scores[0][result]))

In [0]:
# Save to google drive
!cp *.tflite  "/content/drive/My Drive/test_data/number/."

# Download to local
from google.colab import files
files.download( "./conv_mnist_quant.tflite")
files.download( "./conv_mnist_quant_edgetpu.tflite")